# Load and merge the surveys into one

Now comes the fun part. We need to load all the surveys and merge them into a single file/`Dataset`. To do that, we need to make sure:

* Metadata is standard for all surveys and following the [CF conventions](http://cfconventions.org/Data/cf-conventions/cf-conventions-1.8/cf-conventions.html).
* The `Dataset` is in a usable format.
* Coordinates are all WGS84 for easier manipulation.
* We can access the absolute gravity and do our own corrections.

This should be fun...

In [1]:
from pathlib import Path
import datetime
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
from tqdm import tqdm
import pyproj
import pooch

## Inspect one of the datasets

Load one of them and look at the data and metadata that is available.

In [2]:
data_dir = Path("..") / "data" 

In [3]:
xr.open_dataset(data_dir / "000a522322d3c7d2fd97bf91ba7179e6-P198362-point-gravity.nc")

<xarray.Dataset>
Dimensions:                          (elipsoiddatum: 1, ellipsoidhgtaccmethod: 7, ellipsoidhgtmeth: 22, ellipsoidinsthgterrmethod: 7, gndelevaccmethod: 7, gndelevdatum: 9, gndelevmeth: 22, gndelevtype: 12, gravaccmeth: 7, gravmeth: 44, gridflag: 3, insthgterrmeth: 7, insthgtmeth: 22, locaccmethod: 7, locmethod: 13, point: 73, reliab: 10, stattype: 24, tcmeth: 15)
Coordinates:
  * locmethod                        (locmethod) object 'Positions digitised ...
  * locaccmethod                     (locaccmethod) object 'Value taken as re...
  * stattype                         (stattype) object 'Absolute measurement'...
  * gravmeth                         (gravmeth) object 'Pendulum observation ...
  * gravaccmeth                      (gravaccmeth) object 'Value taken as rep...
  * gndelevtype                      (gndelevtype) object 'Marine' ... 'Unkno...
  * gndelevdatum                     (gndelevdatum) object 'Australian Height...
  * gndelevmeth                      (gndelevmeth) object 'Value measured by ...
  * gndelevaccmethod                 (gndelevaccmethod) object 'Value taken a...
  * insthgtmeth                      (insthgtmeth) object 'Value measured by ...
  * insthgterrmeth                   (insthgterrmeth) object 'Value taken as ...
  * ellipsoidinsthgterrmethod        (ellipsoidinsthgterrmethod) object 'Valu...
  * ellipsoidhgtmeth                 (ellipsoidhgtmeth) object 'Value measure...
  * ellipsoidhgtaccmethod            (ellipsoidhgtaccmethod) object 'Value ta...
  * elipsoiddatum                    (elipsoiddatum) object 'GRS80'
  * tcmeth                           (tcmeth) object 'Method developed by Fea...
  * gridflag                         (gridflag) object 'Station not used in t...
  * reliab                           (reliab) object 'Unreliable data which s...
Dimensions without coordinates: point
Data variables:
    crs                              int8 ...
    ga_gravity_metadata              int8 ...
    obsno                            (point) float64 ...
    latitude                         (point) float64 ...
    longitude                        (point) float64 ...
    locmethod_index                  (point) float32 ...
    locacc                           (point) float32 ...
    locaccmethod_index               (point) float32 ...
    stattype_index                   (point) float32 ...
    grav                             (point) float64 ...
    gravmeth_index                   (point) float32 ...
    freeair                          (point) float32 ...
    bouguer                          (point) float32 ...
    gravacc                          (point) float32 ...
    gravaccmeth_index                (point) float32 ...
    gndelev                          (point) float32 ...
    gndelevacc                       (point) float32 ...
    gndelevtype_index                (point) float32 ...
    gndelevdatum_index               (point) float32 ...
    gndelevmeth_index                (point) float32 ...
    gndelevaccmethod_index           (point) float32 ...
    insthgt                          (point) float32 ...
    insthgterr                       (point) float32 ...
    insthgtmeth_index                (point) float32 ...
    insthgterrmeth_index             (point) float32 ...
    ellipsoidinsthgt                 (point) float32 ...
    ellipsoidinsthgterr              (point) float32 ...
    ellipsoidinsthgterrmethod_index  (point) float32 ...
    ellipsoidhgt                     (point) float32 ...
    ellipsoidhgtacc                  (point) float32 ...
    ellipsoidhgtmeth_index           (point) float32 ...
    ellipsoidhgtaccmethod_index      (point) float32 ...
    elipsoiddatum_index              (point) float32 ...
    tc                               (point) float32 ...
    tcdensity                        (point) float32 ...
    tcerr                            (point) float32 ...
    tcmeth_index                     (point) float32 ...
    gridflag_index

## Load and filter surveys

Load all surveys (can take a little while).

In [4]:
all_surveys = [xr.open_dataset(fname) for fname in tqdm(data_dir.glob("*.nc"))]

1631it [01:30, 18.07it/s]


Classify the surveys based on the reliability index (see the metadata in the file above).

In [5]:
reliability = [np.unique(s.reliab_index) for s in all_surveys]

In [6]:
really_bad = [np.any(r == 0) for r in reliability]
bad = [np.any(r == 2) for r in reliability]
not_good = [np.any(r == 3) for r in reliability]
print(f"Really bad: {sum(really_bad)}")
print(f"       Bad: {sum(bad)}")
print(f"  Not good: {sum(not_good)}")

Really bad: 0
       Bad: 32
  Not good: 109


Remove the bad surveys, which seem to not even be recommended for serious use.

In [7]:
surveys = [survey for survey, reliability in zip(all_surveys, reliability) if not np.any(reliability == 2)]
print(len(surveys))

1599


Check the total number of observations left after filtering.

In [8]:
ndata_per_survey = [s.grav.size for s in surveys]
print(sum(ndata_per_survey))

1789824


## Select, convert, and merge

Select only the data fields we're interested in (location, ellipsoid heights, raw gravity, and accuracy measures). 
We'll also convert the datum from GDA94 to WGS84, which is easier to use with most applications.
While we're at it, convert all the data to float32 to save space.

In [9]:
dims = ("point", )
gda_to_wgs = pyproj.Transformer.from_crs("epsg:4283", "epsg:4326", always_xy=True)
datasets = []
for survey in tqdm(surveys, ncols=100):
    # Transform coordinates to WGS84
    lon, lat, h = gda_to_wgs.transform(
        survey.longitude.values, 
        survey.latitude.values, 
        survey.ellipsoidinsthgt.values,
    )
    dataset = xr.Dataset(
        data_vars={
            "gravity": (dims, survey.grav.astype(np.float32) / 10),
            "gravity_accuracy": (dims, survey.gravacc / 10),
            "height_error": (dims, survey.ellipsoidinsthgterr),
            "reliability_index": (dims, survey.reliab_index.astype(np.uint8)),
        },
        coords={
            "longitude": (dims, lon),
            "latitude": (dims, lat),
            "height": (dims, h.astype(np.float32)),
        },
    )
    datasets.append(dataset)

100%|███████████████████████████████████████████████████████████| 1599/1599 [01:11<00:00, 22.23it/s]


Now we can merge the datasets into one and set the metadata for the entire collection.

In [10]:
data = xr.concat(datasets, "point")

data.attrs = {
    "Conventions": "CF-1.8",
    "title": "Compilation of gravity ground surveys for Australia",
    "institution": "Commonwealth of Australia (Geoscience Australia)",
    "crs": "WGS84",
    "source": (
        "Compiled from the collection by Wynne, P. 2018. "
        "NetCDF Ground Gravity Point Surveys Collection. Geoscience Australia, Canberra. "
        "https://doi.org/10.26186/5c1987fa17078 "
    ),    
    "uuid": "d6e3c3a8-5a20-4d8b-afca-e55f754e4ce1",
    "license": "Creative Commons Attribution 4.0 International Licence",
    "references": "https://doi.org/10.26186/5c1987fa17078",   
    "history": (
        f"{datetime.datetime.now().astimezone().isoformat(timespec='seconds')} : "
        "Data with reliability index of 0 or 2 were removed from the compilation. "
        "Coordinates were converted to WGS84. "
        "Gravity was converted to mGal. "
        "Only absolute gravity, position, ellipsoid height and error measures were kept. "
        "Metadata was edited to follow CF conventions more closely. "
    ),     
}
data.gravity.attrs = {
    "long_name": "gravity acceleration",
    "units": "mGal",
    "actual_range": (data.gravity.values.min(), data.gravity.values.max()),
    "ancillary_variables": "gravity_accuracy reliability_index",
    "description": "magnitude of the gravity acceleration vector",
}
data.gravity_accuracy.attrs = {
    "long_name": "accuracy of gravity acceleration",
    "units": "mGal",
    "actual_range": (data.gravity_accuracy.values.min(), data.gravity_accuracy.values.max()),
    "description": "accuracy of the magnitude of the gravity acceleration vector",
}
data.longitude.attrs = {
    "long_name": "longitude",
    "standard_name": "longitude",
    "units": "degrees_east",
    "actual_range": (data.longitude.values.min(), data.longitude.values.max()),
}
data.latitude.attrs = {
    "long_name": "latitude",
    "standard_name": "latitude",
    "units": "degrees_north",
    "actual_range": (data.latitude.values.min(), data.latitude.values.max()),
}
data.height.attrs = {
    "long_name": "geometric height",
    "standard_name": "height_above_reference_ellipsoid",
    "units": "m",
    "actual_range": (data.height.values.min(), data.height.values.max()),
    "description": "height above the WGS84 ellipsoid",
    "ancillary_variables": "height_error",
}
data.height_error.attrs = {
    "long_name": "geometric height error",
    "units": "m",
    "actual_range": (data.height_error.values.min(), data.height_error.values.max()),
    "description": "error in the height above the WGS84 ellipsoid",
}
data.reliability_index.attrs = {
    "long_name": "station reliability",
    "standard_name": "status_flag",    
    "description": "estimate of gravity station reliability",
    "flag_values": np.arange(10, dtype=np.int8),
    "flag_meanings": (
        "unreliable_data_which_should_not_be_used_pending_remedial_action "
        "insufficient_information_to_accurately_classify_but_still_regarded_as_reliable_data "
        "poorly_controlled_data_which_should_be_used_cautiously "
        "data_with_weak_gravity_position_and_elevation_control "
        "data_with_moderate_gravity_position_and_elevation_control "
        "documented_gravity_ties_levelled_elevations_and_accurately_scaled_positions "
        "a_point_occupied_once_with_well_defined_position_and_elevation "
        "multiple_occupations_at_a_point_with_well_defined_position_and_elevation "
        "multiple_measurements_at_a_point_with_accurate_position_and_elevation "
        "data_measured_numerous_times_with_absolute_geodetic_or_first_order_precision"
        ),
}

# Have a look at the compiled Dataset
data

<xarray.Dataset>
Dimensions:            (point: 1789824)
Coordinates:
    longitude          (point) float64 140.1 140.1 140.1 ... 136.0 136.0 136.1
    latitude           (point) float64 -31.56 -31.56 -31.59 ... -33.49 -33.5
    height             (point) float32 111.654 112.496 ... 100.925 144.069
Dimensions without coordinates: point
Data variables:
    gravity            (point) float32 979409.5 979410.1 ... 979583.0 979574.9
    gravity_accuracy   (point) float32 0.05 0.05 0.05 0.05 ... 0.05 0.05 0.05
    height_error       (point) float32 1.0 1.0 1.0 1.0 1.0 ... 2.0 2.0 2.0 2.0
    reliability_index  (point) uint8 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
Attributes:
    Conventions:  CF-1.8
    title:        Compilation of gravity ground surveys for Australia
    institution:  Commonwealth of Australia (Geoscience Australia)
    crs:          WGS84
    source:       Compiled from the collection by Wynne, P. 2018. NetCDF Grou...
    uuid:         d6e3c3a8-5a20-4d8b-afca-e55f754e4ce1
    license:      Creative Commons Attribution 4.0 International Licence
    references:   https://doi.org/10.26186/5c1987fa17078
    history:      2020-10-28T07:21:10+00:00 : Data with reliability index of ...

## Save to netCDF

Export this collection to a file. We'll use integer encoding of some variables to save storage space. 
The errors don't vary largely so we can scale them and store them as 16-bit integers as opposed to 32-bit floats. 
The horizontal coordinates can be stored as 32-bit integers instead of 64-bit floats with roughly centimeter level accuracy. 
Other variables can't be easily compressed this way given their range so we'll leave them as is.

In [11]:
output_file = Path("..") / "australia-ground-gravity.nc"

In [12]:
data.to_netcdf(
    output_file, 
    format="NETCDF4",
    encoding={
        "gravity_accuracy": {'dtype': 'int16', 'scale_factor': 0.0001, '_FillValue': -9_999}, 
        "height_error": {'dtype': 'int16', 'scale_factor': 0.001, '_FillValue': -9_999},
        # Roughly cm level accuracy is stored for the horizontal coordinates
        "latitude": {'dtype': 'int32', 'scale_factor': 1e-07, '_FillValue': -999_999_999},
        "longitude": {'dtype': 'int32', 'scale_factor': 1e-07, '_FillValue': -999_999_999},
    },    
)

Get the SHA256 hash of the data for reference.

In [13]:
print(f"sha256:{pooch.file_hash(output_file)}")

sha256:6b1140998dc15f5e76fc1e5882edff979e993353348cfcfc30a4dfd03c325b24


Load the data back in to check if saving and encoding worked as expected.

In [14]:
data = xr.load_dataset(output_file)
data

<xarray.Dataset>
Dimensions:            (point: 1789824)
Coordinates:
    longitude          (point) float64 140.1 140.1 140.1 ... 136.0 136.0 136.1
    latitude           (point) float64 -31.56 -31.56 -31.59 ... -33.49 -33.5
    height             (point) float32 111.654 112.496 ... 100.925 144.069
Dimensions without coordinates: point
Data variables:
    gravity            (point) float32 979409.5 979410.1 ... 979583.0 979574.9
    gravity_accuracy   (point) float32 0.049999997 0.049999997 ... 0.049999997
    height_error       (point) float32 1.0 1.0 1.0 1.0 1.0 ... 2.0 2.0 2.0 2.0
    reliability_index  (point) uint8 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1 1
Attributes:
    Conventions:  CF-1.8
    title:        Compilation of gravity ground surveys for Australia
    institution:  Commonwealth of Australia (Geoscience Australia)
    crs:          WGS84
    source:       Compiled from the collection by Wynne, P. 2018. NetCDF Grou...
    uuid:         d6e3c3a8-5a20-4d8b-afca-e55f754e4ce1
    license:      Creative Commons Attribution 4.0 International Licence
    references:   https://doi.org/10.26186/5c1987fa17078
    history:      2020-10-28T07:21:10+00:00 : Data with reliability index of ...